## Tabela Silver para Dimenssão de Veiculos

#### Tratamento do Dataframe para obter informações desejadas

Aqui vamos filtrar apenas as colunas desejadas e realizar o tratamentod e tipagem e limpeza de dados necessárias

Vamos realizar alguns tratamentos

    Seleção de colunas necessárias
    Remoção de duplicidades aproveitado já o ID_Veiculo que vem no arquivo original (Por ser o campo unico de cada veiculo já)
    Configurar colunas para o padrão
    Filtrando dados indesejados
    Alteração de tipagem de dados
    Removendo dados nulos
    Salvando arquivo de forma compactada

In [0]:
from pyspark.sql.types import * # Todos as configurações do SQL de tipagem no spark
from pyspark.sql.functions import * # Todas as funções de SQL do Spark
from pyspark.sql.window import Window

In [0]:
# Diretorio Bronze, com os dados brutos das causas dos acidentes
Bronze_Acidentes_Causas = '/Volumes/workspace/dw_acidentes/01_bronze/Acidentes_Causas/'

Diretorio_Silver = '/Volumes/workspace/dw_acidentes/02_Silver'

In [0]:
# Lê os arquivos Parquet da Bronze
df_Acidentes_Causas = spark.read.parquet(Bronze_Acidentes_Causas)

In [0]:
# Selecionado apenas as colunas necessária para criar o modelo Dimenssional de Veiculos
df_veiculos = df_Acidentes_Causas.select(['id_veiculo','tipo_veiculo', 'marca', 'ano_fabricacao_veiculo'])

In [0]:
# Removendo Duplicadas df de veiculos no campo chave
df_veiculos = df_veiculos.dropDuplicates(subset=["id_veiculo"])

In [0]:
# Configurado o nome das colunas para manter um padrão de minuscula e com underscore no lugar de espaçamentos
# Renomear todas as colunas convertendo para minúsculas
df_veiculos = df_veiculos.toDF(*[c.lower() for c in df_veiculos.columns])
# Renomear e substituir espaços por underscore
df_veiculos = df_veiculos.toDF(*[c.replace(' ', '_') for c in df_veiculos.columns])

In [0]:
# Removendo dados Indesejados dos do campo ID_Veiculos
df_veiculos = df_veiculos.where(col('id_Veiculo') != 'NA')

In [0]:
# Realizando transformações de tipagem no arquivo
df_veiculos = df_veiculos.withColumn('id_veiculo', col('id_veiculo').try_cast(IntegerType()))
df_veiculos = df_veiculos.withColumn('ano_fabricacao_veiculo', col('ano_fabricacao_veiculo').try_cast(IntegerType()))

In [0]:
# Retirando os nulos usando como parametro o campo chave pois não pode ser nulo
df_veiculos = df_veiculos.dropna(subset='id_veiculo')

In [0]:
# Validando os campos após os tratamentos realizados
df_veiculos.printSchema()

root
 |-- id_veiculo: integer (nullable = true)
 |-- tipo_veiculo: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- ano_fabricacao_veiculo: integer (nullable = true)



In [0]:
# Avaliando os primeiros registros analise rapida.
display(df_veiculos.limit(5))

id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo
1561,Automóvel,TOYOTA/COROLLA ALTISFLEX,2014
2371,Automóvel,HONDA/CITY DX FLEX,2012
2773,Caminhão,M.BENZ/ATEGO 2425,2008
3727,Motocicleta,HONDA/CG 125 FAN ES,2009
2945,Automóvel,FIAT/UNO WAY 1.4,2013


In [0]:
# Salvando o arquivo em parquet com compressão gzip
df_veiculos.write.parquet(
    f"{Diretorio_Silver}/Dim_Veiculos", 
    mode="overwrite",         # Aqui defino para sempre sobrescrever o arquivo
    compression="gzip"        # Comando para salvar o arquivo com compactação
)